In [1]:
%load_ext autoreload 
%autoreload 2
import argparse
import datetime as dt
from typing import List
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils.data_eng import features_for_model

import pandas as pd 
import numpy as np  
from fundamental_features import *
from market_features import *
from utils.func import *


In [2]:
market_data = pd.read_parquet('data/US/test_mkt_data.pq')
balance_sheet = pd.read_parquet('data/US/test_balance_sheet.pq')
earnings = pd.read_parquet('data/US/test_earnings.pq')
income_statement = pd.read_parquet('data/US/test_income_statement.pq')
cash_flow = pd.read_parquet('data/US/test_cash_flow.pq')
GICS = pd.read_parquet('data/US/test_GICS.pq')
P = pd.read_parquet('data/US/test_adv_table.pq')
P.index.names = ['Date']
R = market_data['close'].unstack().reindex_like(P).pct_change()
market_data.tail(5)

C:\Users\marti\AppData\Local\Temp\ipykernel_24868\3883437686.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  R = market_data['close'].unstack().reindex_like(P).pct_change()


,,datetime,open,high,low,close,adj_close,volume,MktCap
Date,Ticker,,,,,,,,
2024-02-26,ZTS,2024-02-26,196.90,197.99,195.46,197.04,197.04,2098500,NaN
2024-02-27,ZTS,2024-02-27,197.65,199.25,194.04,199.00,199.00,1982500,NaN
2024-02-28,ZTS,2024-02-28,199.30,200.50,197.10,199.94,199.94,2003300,NaN
2024-02-29,ZTS,2024-02-29,200.50,200.53,198.19,198.33,198.33,2849200,2.791120e+12
2024-03-01,ZTS,2024-03-01,196.00,196.23,188.49,190.82,190.82,3706565,2.774288e+12


In [6]:
%load_ext autoreload 
%autoreload 2
from utils.data_eng import features_for_model
close = market_data['adj_close'].unstack()[['AAPL']]
market_features = aggregate_features_model(close)
market_features.columns

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 1/1 [00:00<00:00,  2.15it/s]


Index(['close', 'srs', 'daily_returns', 'daily_vol', 'weekly_returns',
       'monthly_returns', 'quarterly_returns', 'biannual_returns',
       'target_returns', 'daily_vol_21', 'daily_vol_5', 'norm_daily_return',
       'norm_weekly_return', 'norm_monthly_return', 'norm_quarterly_return',
       'norm_biannual_return', 'norm_annual_return', 'beta_252', 'beta_126',
       'rsi_252', 'rsi_126', 'rsi_63', 'rsi_21', 'cti_252', 'cti_126',
       'cti_63', 'cti_21', 'alpha_daily_returns', 'alpha_weekly_returns',
       'alpha_monthly_returns', 'specific_variance', 'total_variance',
       'annual_alpha_variance_ratio', 'macd_5_15', 'macd_10_30', 'macd_21_63',
       'macd_63_126', 'month_of_year', 'year', 'date'],
      dtype='object')

In [9]:
adv = market_data['close'].unstack().shift().rolling(252,min_periods=10).mean() * market_data['volume'].unstack().shift().rolling(252,min_periods=10).mean()
adv = adv.rank(axis=1,ascending=False,pct=False)
new_P = adv<=500
write_to_parquet(new_P,'US','test_adv_table')
new_P

data to parquet done --> data/US/test_adv_table.pq


Ticker,A,AA,AAL,AAP,AAPL,ABBV,ABC,ABK,ABMD,ABT,...,XRX,XTO,XYL,YHOO,YUM,ZBH,ZBRA,ZION,ZMH,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-01-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-01-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-01-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2000-01-07,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,True,True,True,True,True,True,True,False,False,True,...,False,False,True,False,True,True,True,True,False,True
2024-02-27,True,True,True,True,True,True,True,False,False,True,...,False,False,True,False,True,True,True,True,False,True
2024-02-28,True,True,True,True,True,True,True,False,False,True,...,False,False,True,False,True,True,True,True,False,True


In [16]:
MKT_RETURNS = pd.read_csv("data/US/mkt_returns.csv", index_col=0)
MKT_RETURNS

,mkt_returns
2008-02-01,0.020566
2008-02-04,-0.019849
2008-02-05,-0.027790
2008-02-06,-0.010869
2008-02-07,0.020118
...,...
2023-03-31,0.014761
2023-04-03,0.001312
2023-04-04,-0.009895
2023-04-05,-0.002612


In [26]:
mkt_returns = pd.DataFrame(R[new_P].dropna(axis=0,how = 'all').mean(1),columns = ['mkt_returns'])
mkt_returns.index.name = None
mkt_returns.to_csv('data/US/mkt_returns.csv')

In [7]:
fund_data = []
fund_data.append(get_chatoia(income_statement,balance_sheet,P))
# fund_data.append(get_mve(market_data,P))
fund_data.append(get_sue(earnings,P))
fund_data.append(get_turn(market_data,balance_sheet,P))
fund_data.append(get_indmom(R,GICS,P))
fund_data.append(get_grcapx(cash_flow,P))
fund_data.append(get_ill(R,market_data,P))
fund_data.append(get_ibq(earnings,P))
fund_data.append(get_ear(R,earnings,P))
fund_data.append(get_retvol(R,P))
# fund_data.append(get_cash(balance_sheet,P))
fund_data.append(get_agr(balance_sheet,P))
fund_data.append(get_chsho(balance_sheet,P))
fund_data.append(get_chinv(balance_sheet,P))
fund_data.append(get_chtx(income_statement,P))
fund_data.append(get_invest(balance_sheet,P))
fund_data.append(get_currat(balance_sheet,P))


100%|██████████| 12/12 [00:02<00:00,  5.81it/s]
c:\Users\marti\Documents\research perso\fundamental_features.py:60: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  grcaps = grcaps.pct_change(506).clip(-3,3)
c:\Users\marti\Documents\research perso\fundamental_features.py:109: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  agr = total_assets.pct_change(252).clip(-2,2)
c:\Users\marti\Documents\research perso\fundamental_features.py:115: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA val

In [9]:
fundamental_data = pd.concat(fund_data,axis=1)

In [12]:
write_to_parquet(fundamental_data,'US','fundamental_features')

data to parquet done --> data/US/fundamental_features.pq


In [8]:

close = market_data['adj_close'].unstack()
market_features = aggregate_features_model(close)

  1%|          | 7/879 [00:03<05:55,  2.45it/s]c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas_ta\overlap\linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
  5%|▍         | 43/879 [00:18<06:16,  2.22it/s]c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas_ta\overlap\linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas_ta\overlap\linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
  6%|▋         | 56/879 [00:24<06:13,  2.21it/s]c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas_ta\overlap\linreg.py:53: RuntimeWarning: invalid value encountered in scalar divide
  return rn / rd
c:\Users\marti\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas_ta\overlap\linreg.py:53: RuntimeWarning: invalid valu

In [9]:
write_to_parquet(market_features,'US','market_features')

data to parquet done --> data/US/market_features.pq
